<a href="https://colab.research.google.com/github/lonecoder93/Rice-Leaf-Disease-Detection/blob/main/Rice_Leaf_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Problem Statement

Create a model which can classify the three major attacking diseases of rice plants like leaf smut, bacterial blight and brown spot.

In [ ]:
# Connecting to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Changing directory to Google Drive
%cd /content/gdrive

/content/gdrive


In [ ]:
# importing libraries & Packages
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

-----------------------------------------------------------------------------------------------------------

# CNN Architecture
datagen = ImageDataGenerator(
    rotation_range=90, width_shift_range=0.2, height_shift_range=0.2,
    rescale=1./255, shear_range=0.5, zoom_range=0.5, horizontal_flip=True, fill_mode="nearest")

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

-----------------------------------------------------------------------------------------------------------

# Flattening
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(3))  # Number of output neurons is 3
model.add(Activation('softmax'))

-----------------------------------------------------------------------------------------------------------

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

-----------------------------------------------------------------------------------------------------------

# Train Test Split
batch_size = 5

train_datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Rice Leaf/Train', target_size=(150, 150), batch_size=batch_size, class_mode='categorical', shuffle=True)

validation_generator = test_datagen.flow_from_directory('/content/gdrive/MyDrive/Rice Leaf/Test', target_size=(150, 150), batch_size=batch_size, class_mode='categorical')

-----------------------------------------------------------------------------------------------------------

# Model Fitting
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)


Found 105 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Epoch 1/50
21/21 [==============================] - 39s 2s/step - loss: 1.1065 - accuracy: 0.3524 - val_loss: 1.0983 - val_accuracy: 0.4667
Epoch 2/50
21/21 [==============================] - 5s 253ms/step - loss: 1.0866 - accuracy: 0.4095 - val_loss: 1.0873 - val_accuracy: 0.4000
Epoch 3/50
21/21 [==============================] - 7s 351ms/step - loss: 1.0471 - accuracy: 0.5714 - val_loss: 1.0397 - val_accuracy: 0.4000
Epoch 4/50
21/21 [==============================] - 5s 239ms/step - loss: 0.9993 - accuracy: 0.5333 - val_loss: 1.1543 - val_accuracy: 0.4000
Epoch 5/50
21/21 [==============================] - 7s 354ms/step - loss: 0.9746 - accuracy: 0.6000 - val_loss: 1.1256 - val_accuracy: 0.4000
Epoch 6/50
21/21 [==============================] - 5s 257ms/step - loss: 0.8787 - accuracy: 0.5905 - val_loss: 1.1545 - val_accuracy: 0.4000
Epoch 7/50
21/21 [==============================] - 9s 429ms/step - l

In [ ]:
# Saving pre-trained model
model.save("/content/gdrive/MyDrive/Rice Leaf/trained_model_1.h5")

In [ ]:
# Testing the model

from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img


import numpy as np

# Load the trained model
model = load_model("/content/gdrive/MyDrive/Rice Leaf/trained_model.h5")

# Load the image. This image for Bacterical leaf blight has been downloaded from google with more complex background
img = load_img("/content/gdrive/MyDrive/Validation/Bacterial leaf blight.jpg")

# Resize the image to the desired input size
x = img_to_array(img.resize([150, 150]))

# Expand dimensions to match the expected input shape of the model (1, 150, 150, 3)
x = np.expand_dims(x, axis=0)

# Replace 'model' with the name of your trained model variable
predictions = model.predict(x)

# Get the predicted class index (0, 1, or 2) with the highest probability
predicted_class_index = np.argmax(predictions)

# Map the class index to class labels (modify as per your class labels)
class_labels = ["Bacteral leaf blight", "brown spot", "leaf smut"]
predicted_class_label = class_labels[predicted_class_index]

print("Predicted Class:", predicted_class_label)

1/1 [==============================] - 0s 154ms/step
Predicted Class: Bacteral leaf blight


In [ ]:
# Load the image. This image for brown spot has been downloaded from google with more complex background
img = load_img("/content/gdrive/MyDrive/Validation/Brown Spot.JPG")

print("Predicted Class:", predicted_class_label)

Predicted Class: brown spot


In [ ]:
# Load the image. This image for brown spot has been downloaded from google with more complex background
img = load_img("/content/gdrive/MyDrive/Validation/leaf smut.jpg")

print("Predicted Class:", predicted_class_label)

Predicted Class: brown spot


# Conclusion
The model accurately identifies bacterial leaf blight and brown spot. However, when it comes to leaf smut, it tends to classify it as brown spot. This could be due to the limited visual distinction between brown spot and leaf smut and very small amount of data available.